In [1]:
import os
import numpy as np
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.svm import OneClassSVM
import joblib
import matplotlib.pyplot as plt

In [2]:
# Set paths for training and test data
train_dir = r'D:\Onedrive\Desktop\L6\fruits-360-original-size\Training'
test_dir = r'D:\Onedrive\Desktop\L6\fruits-360-original-size\Test'

In [3]:
# DenseNet201 model setup
feature_extractor = DenseNet201(weights='imagenet', include_top=False, pooling='avg')
feature_extractor.trainable = False  # Freeze the model

In [5]:
def extract_features(image_path):
    """Extract features for a single image."""
    img = load_img(image_path, target_size=(224, 224))  # Resize to match DenseNet input
    img_array = img_to_array(img) / 255.0  # Normalize image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    features = feature_extractor.predict(img_array)  # Get DenseNet features
    return features.flatten()  # Flatten the features to 1D array

In [6]:
# Extract features from all 15 folders of apple images
apple_folders = ['apple_6', 'apple_braeburn_1', 'apple_crimson_snow_1', 'apple_golden_1',
                 'apple_golden_2', 'apple_golden_3', 'apple_granny_smith_1', 'apple_hit_1',
                 'apple_pink_lady_1', 'apple_red_1', 'apple_red_2', 'apple_red_3',
                 'apple_red_delicios_1', 'apple_red_yellow_1', 'apple_rotten_1']  # Replace with actual folder names if different

apple_features = []

for folder_name in apple_folders:
    folder_path = os.path.join(train_dir, folder_name)  # Combine base path with folder name
    print(f"Processing folder: {folder_name}")

    for img_name in os.listdir(folder_path):
        if img_name.endswith(('jpg', 'jpeg', 'png')):  # Check for valid image files
            img_path = os.path.join(folder_path, img_name)
            try:
                features = extract_features(img_path)
                apple_features.append(features)
            except Exception as e:
                print(f"Error processing {img_path}: {e}")

Processing folder: apple_6
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1

In [7]:
# Convert to NumPy array and save
apple_features = np.array(apple_features)
np.save("apple_features.npy", apple_features)
print(f"Extracted features from {len(apple_features)} apple images across all folders.")

Extracted features from 4821 apple images across all folders.


In [8]:
# Train One-Class SVM on all apple features
svm_model = OneClassSVM(kernel='rbf', gamma='auto').fit(apple_features)

In [10]:
# Save the trained SVM model
joblib.dump(svm_model, r"D:\Onedrive\Desktop\L6 II Apple\flask_app\models\one_class_svm_model.joblib")
print("One-Class SVM model trained and saved successfully.")

One-Class SVM model trained and saved successfully.
